In [1]:
import cv2
import numpy as np
import datetime

In [2]:
cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read('trained_model.xml')

In [3]:
cap = cv2.VideoCapture(0)

def detect_faces_and_recognize(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detecting faces using the Haar Cascade
    faces = cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

        # Region of interest (ROI) for face recognition
        roi_gray = gray[y:y + h, x:x + w]
        
        # Perform face recognition using LBPH
        label, confidence = recognizer.predict(roi_gray)
        
        # Display the label and confidence on the frame
        cv2.putText(frame, f'ID: {label} Conf: {confidence:.2f}', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    
    return frame

# main loop to process the video feed
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break
    
    frame = detect_faces_and_recognize(frame)

    cv2.imshow('Video Feed', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#cap.release()
#cv2.destroyAllWindows()

In [4]:
# Current effect
current_effect = 'None'

# Applying effects
def apply_effect(frame, effect):
    global current_effect
    if effect == 'flip_vertical':
        frame = cv2.flip(frame, 0)
    elif effect == 'flip_horizontal':
        frame = cv2.flip(frame, 1)
    elif effect == 'rotate':
        frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    elif effect == 'grayscale':
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frame = cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR) 
    elif effect == 'hsv':
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    elif effect == 'lab':
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2LAB)
    elif effect == 'blur':
        frame = cv2.GaussianBlur(frame, (15, 15), 0)
    elif effect == 'median':
        frame = cv2.medianBlur(frame, 15)
    elif effect == 'draw_shapes':
        # Drawing a rectangle and a circle on the frame
        cv2.rectangle(frame, (50, 50), (200, 200), (0, 255, 0), 2)
        cv2.circle(frame, (300, 300), 50, (255, 0, 0), 2)
    return frame

In [5]:
#main

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = apply_effect(frame, current_effect)
    frame = detect_faces_and_recognize(frame)
    current_time = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    cv2.putText(frame, f'Time: {current_time}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow('Video', frame)
    
    # Key events
    key = cv2.waitKey(1) & 0xFF
    if key == ord('v'):  # flip vertically
        current_effect = 'flip_vertical'
    elif key == ord('h'):  # flip horizontally
        current_effect = 'flip_horizontal'
    elif key == ord('r'):  # rotate
        current_effect = 'rotate'
    elif key == ord('g'):  # grayscale
        current_effect = 'grayscale'
    elif key == ord('s'):  # switch to HSV
        current_effect = 'hsv'
    elif key == ord('l'):  # switch to LAB
        current_effect = 'lab'
    elif key == ord('b'):  # apply Blur
        current_effect = 'blur'
    elif key == ord('n'):  # apply median filter
        current_effect = 'median'
    elif key == ord('d'):  # draw shapes
        current_effect = 'draw_shapes'
    elif key == ord('q'):  # quit
        break

cap.release()
cv2.destroyAllWindows()